# This Assumes PEAS is on Branch<br>`origin/SUR-7006-Modify-Emission-Rate-Calculation-at-the-LISA-Level`

In [1]:
%load_ext lab_black
%load_ext dotenv
%dotenv
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from psanalytics.dto.Peak import Peak
from psanalytics.dto.EmissionSource import EmissionSource
import matplotlib.pyplot as plt
from datetime import datetime
import glob
from common import (
    create_peak_query,
    query_mssql_iteratively,
    get_measurements,
    create_survey_query,
)
from config import CUSTOMER

pd.options.mode.chained_assignment = None
plt.rcParams.update({"font.size": 14})

In [2]:
CUSTOMER

'italgas'

In [3]:
%%time

def compute_source_emission_rate(group):   
    em_temp = []
    peaks = []
    j = 0
    for i in range(len(group)):
        plume_emission_rate = group.iloc[i].PlumeEmissionRate_CORRECTED
        peak = Peak(
            PlumeEmissionRate=plume_emission_rate,
            #PlumeEmissionRateUncertainty=group.iloc[i].PlumeEmissionRateUncertainty,
        )
        peaks.append(peak)
        
    emission_source = EmissionSource(
        peaks=peaks,
    )
        
    j += 1
    emission_source.calc_rate_stats_from_measurements()
    
    emission_source.calc_emission_rate_prob_log_normal()
    em_source = emission_source.EmissionRate

  
    
    
    #print('"EmissionRate": ', emission_source.EmissionRate, ',')
    # print('"EmissionRateAMean": ', emission_source.EmissionRateAMean, ',')
    # print('"EmissionRateAStd": ', emission_source.EmissionRateAStd, ',')
    # print('"EmissionRateGMean": ', emission_source.EmissionRateGMean, ',')
    # print('"EmissionRateGStd": ', emission_source.EmissionRateGStd, ',')
    # print('"EmissionRateLowerBound": ', emission_source.EmissionRateLowerBound, ',')
    # print('"EmissionRateUpperBound": ', emission_source.EmissionRateUpperBound, ',')
    return em_source
 
    

# recalculated_new_algorithm_emission_rates = peaks.groupby("EmissionSourceId").apply(lambda group: compute_source_emission_rate(group, new_algorithm=True))
# recalculated_new_algorithm_emission_rates.head()

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 9.54 µs


## Get the emission sources and Peak Data

In [14]:
emission_sources = pd.read_pickle(
    f"data/non-reprocessed-emission-sources-{CUSTOMER}-errors.pickle"
)
emission_sources.info()
emission_sources.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53 entries, 0 to 59
Data columns (total 22 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   EmissionSourceId          53 non-null     object        
 1   RepresentativePeakId      53 non-null     object        
 2   SurveyId                  53 non-null     object        
 3   AnalyzerId                53 non-null     object        
 4   ReportId                  53 non-null     object        
 5   ReportTitle               53 non-null     object        
 6   DispositionName           53 non-null     object        
 7   Disposition               53 non-null     int64         
 8   PeakNumber                53 non-null     int64         
 9   PriorityScore             53 non-null     float64       
 10  MeasuredSCFH              53 non-null     float64       
 11  MaxAmplitude              53 non-null     float64       
 12  GpsLatitude             

,EmissionSourceId,RepresentativePeakId,SurveyId,AnalyzerId,ReportId,ReportTitle,DispositionName,Disposition,PeakNumber,PriorityScore,...,GpsLatitude,GpsLongitude,CH4,EthaneRatio,EthaneRatioUncertainty,ClassificationConfidence,DateReportStarted,SurveyEndDateTime,Year,Customer
0,d101921a-9d50-41f1-acb5-cb37fba4911d,68801c35-1d78-4be0-a4d7-16727c7ec31b,0e536210-cf55-deaf-b368-3a041abcb199,cbc7fe42-1805-9844-cfd0-39ea78c2d4f2,763dcc0b-0ec4-7d54-0526-3a0458f53adf,Raviscanina 27052022 FINALE,Possible_Natural_Gas,3,4,0.071381,...,41.369283,14.244677,2.361322,-0.023885,0.003335,0.100000,2022-06-09 04:45:46.593,2022-05-28 03:53:57.133,2022,ITALGAS
1,e23a6a59-f164-42c9-8861-9dc90450e538,5d05aa6f-5b83-4ced-a344-5175fe36be35,0e536210-cf55-deaf-b368-3a041abcb199,cbc7fe42-1805-9844-cfd0-39ea78c2d4f2,763dcc0b-0ec4-7d54-0526-3a0458f53adf,Raviscanina 27052022 FINALE,Possible_Natural_Gas,3,0,0.038491,...,41.368056,14.245178,2.167646,0.017163,0.006555,0.779926,2022-06-09 04:45:46.593,2022-05-28 03:53:57.133,2022,ITALGAS
2,aa92608b-51d9-4ca8-9bf5-ab16f74ca302,356b7838-2eda-4519-a617-637b0d17e707,0e536210-cf55-deaf-b368-3a041abcb199,cbc7fe42-1805-9844-cfd0-39ea78c2d4f2,763dcc0b-0ec4-7d54-0526-3a0458f53adf,Raviscanina 27052022 FINALE,Natural_Gas,1,0,0.022525,...,41.371216,14.244913,2.031602,0.081844,0.012183,0.950000,2022-06-09 04:45:46.593,2022-05-28 03:53:57.133,2022,ITALGAS
3,113c2f30-f5d7-466d-8b2d-3d6f7b91c59e,f2577490-cead-4abe-ae93-0b772c48841e,e64205d7-929f-86dd-05cf-3a04153136d3,cbc7fe42-1805-9844-cfd0-39ea78c2d4f2,763dcc0b-0ec4-7d54-0526-3a0458f53adf,Raviscanina 27052022 FINALE,Possible_Natural_Gas,3,0,0.015489,...,41.363401,14.237046,2.110485,0.005088,0.012574,0.107707,2022-06-09 04:45:46.593,2022-05-27 02:20:35.060,2022,ITALGAS
4,f97ecf9b-9ddb-40e1-b12f-dd89f4b32640,a8257b86-5fae-4a32-a454-18edb474bc5a,e64205d7-929f-86dd-05cf-3a04153136d3,cbc7fe42-1805-9844-cfd0-39ea78c2d4f2,763dcc0b-0ec4-7d54-0526-3a0458f53adf,Raviscanina 27052022 FINALE,Possible_Natural_Gas,3,3,0.130839,...,41.361922,14.254904,2.073461,0.036305,0.024783,0.789053,2022-06-09 04:45:46.593,2022-05-27 02:20:35.060,2022,ITALGAS


In [15]:
peaks_csv = pd.read_csv(f"wind_speed_rotations/corrected-peak-emissions-{CUSTOMER}.csv")
peaks_csv["EmissionSourceId"] = peaks_csv[
    "EmissionSourceId"
].str.lower()  # this is because the merging is case sensitive
peaks_csv.info()
peaks_csv.head()

emission_source_ids = (
    peaks_csv.EmissionSourceId.drop_duplicates()
)  # Here we get the emission source id of the peaks that have been corrected. This will be used to get the peaks of these corrections and recalculate the emissions

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144630 entries, 0 to 144629
Data columns (total 29 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Unnamed: 0.1                 144630 non-null  int64  
 1   Unnamed: 0                   144630 non-null  int64  
 2   PeakId                       144630 non-null  object 
 3   SurveyId                     144630 non-null  object 
 4   AnalyzerId                   144630 non-null  object 
 5   EmissionSourceId             144630 non-null  object 
 6   PlumeEpochStart              144630 non-null  float64
 7   PlumeEpochEnd                144630 non-null  float64
 8   LineIntegral                 144630 non-null  float64
 9   Distance                     144630 non-null  float64
 10  Sigma                        144630 non-null  float64
 11  EpochTime                    144630 non-null  float64
 12  Amplitude                    144630 non-null  float64
 13 

peaks = query_mssql_iteratively(emission_source_ids, create_peak_query)

peaks.info()
peaks.head()

In [ ]:
%%time
result = peaks_csv.groupby(by=["EmissionSourceId"], as_index=True).apply(
    compute_source_emission_rate
)

In [ ]:
result.to_csv("data/result_up_to_may_31.csv")

In [7]:
result = pd.read_csv("data/result_up_to_may_31.csv")

## Checking number of anomalies in case you see from the next step that there are som

In [19]:
anomalies = result[result.isna()]
list_emsourceid = anomalies.index.values.tolist()
list_emsourceid

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [18]:
for anomaly_emsourceid in list_emsourceid:
    num_peaks = (
        peaks_csv.query("EmissionSourceId == @anomaly_emsourceid")
        .groupby(["EmissionSourceId"])
        .size()
    )
    # num_peaks_count = num_peaks[0]
    # print(num_peaks_count)

In [20]:
num_peaks

Series([], dtype: int64)

In [ ]:
n_sample = 40
for anomaly_emsourceid in list_emsourceid:
    print(anomaly_emsourceid)
    anomaly = (
        peaks_csv.query("EmissionSourceId == @anomaly_emsourceid")
        .sample(n=n_sample)
        .groupby(by=["EmissionSourceId"], as_index=True)
        .apply(compute_source_emission_rate)
    )
    print(anomaly)

    if anomaly[0] == np.nan:
        print("EmissionSourceId '{}' has an issue".format(anomaly_emsourceid))
        break
    else:
        print("ok")
        print("anomalies")
        result.loc[anomaly_emsourceid] = anomaly[0]

In [ ]:
result.loc[anomaly_emsourceid]

result.loc[anomaly_emsourceid]

peaks_csv.query("EmissionSourceId == '645bb627-6311-4f63-8855-668d34de0ba2'").count()


%%time
anomaly = peaks_csv.query("EmissionSourceId == '645bb627-6311-4f63-8855-668d34de0ba2'").sample(n=65).groupby(by=["EmissionSourceId"], as_index=True).apply(
    compute_source_emission_rate
)

In [21]:
anomaly = (
    peaks_csv.query("EmissionSourceId == '645bb627-6311-4f63-8855-668d34de0ba2'")
    .groupby(by=["EmissionSourceId"], as_index=True)
    .apply(compute_source_emission_rate)
)

/home/ubuntu/picarro-repos/PEAS/psanalytics_build/core/psanalytics/dto/EmissionSource.py:154: RuntimeWarning: divide by zero encountered in power
  emission_std_mult_sum_sq_inv = sum(emission_rate_uncertainties ** -2)
/home/ubuntu/picarro-repos/PEAS/pseq_build/core/pseq/algorithms/SourceEmissionRate.py:384: RuntimeWarning: overflow encountered in double_scalars
  (emission_rate / geometric_mean) ** alpha * eta ** (-nu) * kv(nu, alpha * eta)
/home/ubuntu/picarro-repos/PEAS/pseq_build/core/pseq/algorithms/SourceEmissionRate.py:384: RuntimeWarning: invalid value encountered in double_scalars
  (emission_rate / geometric_mean) ** alpha * eta ** (-nu) * kv(nu, alpha * eta)
/home/ubuntu/picarro-repos/PEAS/pseq_build/core/pseq/algorithms/SourceEmissionRate.py:156: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  integral = quad(
/home/ubuntu/picarro-repos/PEAS/pseq_build/core

result.loc["645bb627-6311-4f63-8855-668d34de0ba2"] = anomaly

In [ ]:
result_df = pd.DataFrame(result, columns=["EmissionRate"]).reset_index()
result_df["EmissionSourceId"] = result_df["EmissionSourceId"].str.lower()
result_df.info()
result_df.head()

In [ ]:
result_final = pd.merge(
    emission_sources, result_df, on="EmissionSourceId", validate="one_to_one"
)

In [ ]:
result_final.info()

In [ ]:
result_final_save = result_final.drop("MeasuredSCFH", axis=1)
result_final_save.rename(columns={"EmissionRate": "MeasuredSCFH"}, inplace=True)
result_final_save.info()

In [ ]:
result_final_save.to_pickle(
    f"data/non-reprocessed-emission-sources-recalculated{CUSTOMER}-errors.pickle"
)